### Extract Comments and Video Information

In [9]:
import os
from dotenv import load_dotenv
import sys
sys.path.append('src')
import googleapiclient.discovery
import googleapiclient.errors
from ETL.extract.fetch_comments import getYoutubeComments
from ETL.extract.fetch_video_information import getVideos

api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = os.getenv("API_KEY")

video_IDs = ['bMnU3V89jj4']
#'zxYjTTXc-J8', 'fMfipiV_17o', 'FM7Z-Xq8Drc', '1WEAJ-DFkHE', 'r7zJ8srwwjk', 'iogcY_4xGjo', '0e3GPea1Tyg','48h57PspBec','GLoeAJUcz38','9bqk6ZUsKyA'

youtube = googleapiclient.discovery.build(
    api_service_name,
    api_version,
    developerKey=DEVELOPER_KEY
)

youtubeVideoInfo = getVideos(youtube, video_IDs)

youtubeComments = []
for ID in video_IDs:
    youtubeComments.extend(getYoutubeComments(youtube, ID))


In [10]:
from ETL.transform.preprocess import toDataframeComments
from ETL.transform.preprocess import toDataFrameVideoInfo

videoinfo_df = toDataFrameVideoInfo(youtubeVideoInfo)
all_comments_df = toDataframeComments(youtubeComments)

In [11]:
all_comments_df.head()

,Kind,Etag,Comment_ID,Channel_ID,Video_ID,Comment_Text,Original_Comment_Text,Auther_Name,Auther_Profile_Image_URL,Auther_Channel_URL,Auther_Channel_ID,canRate,Viewer_Rate,Comment_Like_Count,Comment_Published_Date,Comment_Updated_Date,Comment_Parent_ID
0,youtube#comment,2TSJd0EvtmDZt_y8dWCqVv_zcOc,UgyRLpx8l3N4pkd01gV4AaABAg,UCoLrcjPV5PbUrUyXq5mjc_A,bMnU3V89jj4,Yo the MLB should stop glazing Shohei like a ...,Yo the MLB should stop glazing Shohei like a ...,@12prodzz,https://yt3.ggpht.com/rFKhPSrQHFVFMiKown24JoXC...,http://www.youtube.com/@12prodzz,UCvX7v_ZhiqQjbHBNUdKkPwA,True,none,0.0,2025-03-02 13:45:34+00:00,2025-03-02 13:46:10+00:00,NaN
1,youtube#comment,rW7mNti9vNJcUB3E_2nsFGJix1I,UgwBWQhnVnbF9qi4OyN4AaABAg,UCoLrcjPV5PbUrUyXq5mjc_A,bMnU3V89jj4,DODGERS RULE 💙💙💙💙,DODGERS RULE 💙💙💙💙,@SwaggerBangger,https://yt3.ggpht.com/YXEJjy1-0zvkgsRsiOU-eJaF...,http://www.youtube.com/@SwaggerBangger,UCFqHmjLXvS4idHsBfgdiFSQ,True,none,1.0,2025-02-26 19:59:12+00:00,2025-02-26 19:59:12+00:00,NaN
2,youtube#comment,S66vW02VFCZxYFzFItvofUKdm6E,Ugxz2qghhiH9ox8O-BJ4AaABAg,UCoLrcjPV5PbUrUyXq5mjc_A,bMnU3V89jj4,DODGERS DOMINATED 2 NY TEAMS METS &amp; YANKEES,DODGERS DOMINATED 2 NY TEAMS METS & YANKEES,@SwaggerBangger,https://yt3.ggpht.com/YXEJjy1-0zvkgsRsiOU-eJaF...,http://www.youtube.com/@SwaggerBangger,UCFqHmjLXvS4idHsBfgdiFSQ,True,none,1.0,2025-02-26 19:58:50+00:00,2025-02-26 19:58:50+00:00,NaN
3,youtube#comment,amXGOQxu0V6UBr-TtlyFO1vSXUE,UgwLniCqHW-ehsVCq3B4AaABAg,UCoLrcjPV5PbUrUyXq5mjc_A,bMnU3V89jj4,Boring game 😂😂😂,Boring game 😂😂😂,@officialcrickethighligh,https://yt3.ggpht.com/hVJUFIBLDRm0_LG0owJ-k-DB...,http://www.youtube.com/@officialcrickethighligh,UCBApcKt34BYqVuV8GN1Kg-g,True,none,0.0,2025-02-05 03:53:18+00:00,2025-02-05 03:53:18+00:00,NaN
4,youtube#comment,pugsK6ua-L5vccEVQHgi6EIUCro,UgyCicep-ndIIZPCwzZ4AaABAg,UCoLrcjPV5PbUrUyXq5mjc_A,bMnU3V89jj4,Maton sucked every single outing 😂,Maton sucked every single outing 😂,@BeerCanDan93,https://yt3.ggpht.com/UriGrsv8NyJ4alkd84Ocw5P4...,http://www.youtube.com/@BeerCanDan93,UCXjIrmEgeqxxraVxTwH8i_A,True,none,0.0,2025-01-18 09:45:28+00:00,2025-01-18 09:45:28+00:00,NaN


In [12]:
videoinfo_df.head()

,Video_ID,Publisded_Time,Channel_ID,Video_Title,Video_View_Count,Video_Like_Count,Video_Comment_Count
0,bMnU3V89jj4,2024-10-21T14:35:11Z,UCoLrcjPV5PbUrUyXq5mjc_A,The Dodgers are WORLD SERIES BOUND! (Full 2024...,299448.0,2403.0,125.0


### Preporcessing, lowercasae, remove stopwords, punctuations, and tokenize

In [13]:
from ETL.transform.preprocess import preprocessText

all_comments_df_preprocessed = preprocessText(all_comments_df, 'Original_Comment_Text')

Error detecting language: No features in text.
Error detecting language: No features in text.
Error detecting language: No features in text.
Error detecting language: No features in text.
Error detecting language: No features in text.


/Users/david/Desktop/University of Sheffield/Data Science/Dissertation - 68/Data & Code/Sentiment Analysis - Youtube comments/src/ETL/transform/preprocess.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['No_Punctuation'] = df['Lowercase'].str.translate(str.maketrans('', '', string.punctuation)) # remove punctuation
/Users/david/Desktop/University of Sheffield/Data Science/Dissertation - 68/Data & Code/Sentiment Analysis - Youtube comments/src/ETL/transform/preprocess.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [14]:
all_comments_df_preprocessed.head()

,Kind,Etag,Comment_ID,Channel_ID,Video_ID,Comment_Text,Original_Comment_Text,Auther_Name,Auther_Profile_Image_URL,Auther_Channel_URL,...,Viewer_Rate,Comment_Like_Count,Comment_Published_Date,Comment_Updated_Date,Comment_Parent_ID,Lowercase,No_Punctuation,Tokenized,No_Stopwords_Tokenized,No_Stopwords_Text
0,youtube#comment,2TSJd0EvtmDZt_y8dWCqVv_zcOc,UgyRLpx8l3N4pkd01gV4AaABAg,UCoLrcjPV5PbUrUyXq5mjc_A,bMnU3V89jj4,Yo the MLB should stop glazing Shohei like a ...,Yo the MLB should stop glazing Shohei like a ...,@12prodzz,https://yt3.ggpht.com/rFKhPSrQHFVFMiKown24JoXC...,http://www.youtube.com/@12prodzz,...,none,0.0,2025-03-02 13:45:34+00:00,2025-03-02 13:46:10+00:00,NaN,yo the mlb should stop glazing shohei like a ...,yo the mlb should stop glazing shohei like a ...,"[yo, the, mlb, should, stop, glazing, shohei, ...","[yo, mlb, stop, glazing, shohei, like, donut]",yo mlb stop glazing shohei like donut
4,youtube#comment,pugsK6ua-L5vccEVQHgi6EIUCro,UgyCicep-ndIIZPCwzZ4AaABAg,UCoLrcjPV5PbUrUyXq5mjc_A,bMnU3V89jj4,Maton sucked every single outing 😂,Maton sucked every single outing 😂,@BeerCanDan93,https://yt3.ggpht.com/UriGrsv8NyJ4alkd84Ocw5P4...,http://www.youtube.com/@BeerCanDan93,...,none,0.0,2025-01-18 09:45:28+00:00,2025-01-18 09:45:28+00:00,NaN,maton sucked every single outing 😂,maton sucked every single outing 😂,"[maton, sucked, every, single, outing]","[maton, sucked, every, single, outing]",maton sucked every single outing
5,youtube#comment,FM3kAT-b9U0VUBV4kBSNTihZkso,UgzvfQQwsYqveadG9Y94AaABAg,UCoLrcjPV5PbUrUyXq5mjc_A,bMnU3V89jj4,😍😍😍&quot;Dodgers outscored the Mets 23-0&quot;...,"😍😍😍""Dodgers outscored the Mets 23-0"" . That's ...",@seun9957,https://yt3.ggpht.com/ytc/AIdro_meLtS1C8gl1i7c...,http://www.youtube.com/@seun9957,...,none,0.0,2024-12-23 04:33:49+00:00,2024-12-23 04:33:49+00:00,NaN,"😍😍😍""dodgers outscored the mets 23-0"" . that's ...",😍😍😍dodgers outscored the mets 230 thats how y...,"[dodgers, outscored, the, mets, 230, thats, ho...","[dodgers, outscored, mets, 230, thats, win, wo...",dodgers outscored mets 230 thats win world ser...
7,youtube#comment,rdfhGQTk_oqna7j818gwKE8uo6k,Ugx4z3EZQS0htKNWTA94AaABAg,UCoLrcjPV5PbUrUyXq5mjc_A,bMnU3V89jj4,"This was a fun series, reminded me of old scho...","This was a fun series, reminded me of old scho...",@decibellx2194,https://yt3.ggpht.com/HEOskTjEef82vv0JKCuXlMYC...,http://www.youtube.com/@decibellx2194,...,none,0.0,2024-11-24 01:52:51+00:00,2024-11-24 01:52:51+00:00,NaN,"this was a fun series, reminded me of old scho...",this was a fun series reminded me of old schoo...,"[this, was, a, fun, series, reminded, me, of, ...","[fun, series, reminded, old, school, baseball]",fun series reminded old school baseball
8,youtube#comment,m2B-JCJ8KL6VNfPPTYcm6-Ni4NM,Ugwjom7jkDqGYCHct3p4AaABAg,UCoLrcjPV5PbUrUyXq5mjc_A,bMnU3V89jj4,The Mets were no match for the Dodgers. Just ...,The Mets were no match for the Dodgers. Just ...,@sandyhanson6082,https://yt3.ggpht.com/ytc/AIdro_m8XKHqSWkWTWqW...,http://www.youtube.com/@sandyhanson6082,...,none,1.0,2024-11-11 05:51:38+00:00,2024-11-11 05:51:38+00:00,NaN,the mets were no match for the dodgers. just ...,the mets were no match for the dodgers just a...,"[the, mets, were, no, match, for, the, dodgers...","[mets, match, dodgers, better, team]",mets match dodgers better team


In [15]:
all_comments_df_preprocessed.shape

(67, 22)

### Sentiment Analysis

In [16]:
from ETL.transform.sentiment_analysis import sentiment_analysis

sentiment_result = sentiment_analysis(all_comments_df_preprocessed, 'Original_Comment_Text')

In [17]:
sentiment_result.head()

,Positive_Score,Neutral_Score,Negative_Score,Compound_Score,Sentiment_Label,Original_Comment_Text,Comment_ID
0,0.197,0.630,0.173,0.0772,Positive,Yo the MLB should stop glazing Shohei like a ...,UgyRLpx8l3N4pkd01gV4AaABAg
4,0.242,0.446,0.312,-0.0258,Neutral,Maton sucked every single outing 😂,UgyCicep-ndIIZPCwzZ4AaABAg
5,0.432,0.568,0.000,0.9432,Positive,"😍😍😍""Dodgers outscored the Mets 23-0"" . That's ...",UgzvfQQwsYqveadG9Y94AaABAg
7,0.248,0.752,0.000,0.5106,Positive,"This was a fun series, reminded me of old scho...",Ugx4z3EZQS0htKNWTA94AaABAg
8,0.192,0.662,0.146,0.1779,Positive,The Mets were no match for the Dodgers. Just ...,Ugwjom7jkDqGYCHct3p4AaABAg


### Topic Modelling

In [18]:
from ETL.transform.topic_modelling import create_bertopic_model

model, topics, probabilities, topic_modelling_df = create_bertopic_model(
    all_comments_df_preprocessed, 'No_Stopwords_Text', 'Comment_ID', 15
)

/Users/david/Desktop/University of Sheffield/Data Science/Dissertation - 68/Data & Code/Sentiment Analysis - Youtube comments/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-06 16:18:56,740 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 3/3 [00:01<00:00,  2.52it/s]
2025-03-06 16:19:01,126 - BERTopic - Embedding - Completed ✓
2025-03-06 16:19:01,126 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2025-03-06 16:19:03,704 - BERTopic - Dimensionality - Completed ✓
2025-03-06 16:19:03,705 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-03-06 16:19:03,711 - BERTopic - Cluster - Completed ✓
2025-03-06 16:19:03,7

In [19]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,33,-1_ws_mets_love_contracts,"[ws, mets, love, contracts, baseball, fan, pad...",[heberenos5970i think meant mets made nlcs pad...
1,0,10,0_ohtani_second_see_japanese,"[ohtani, second, see, japanese, korea, blind, ...",[hhattonaom9729 ohtani outperforming lindor ev...
2,1,24,1_dodgers_go_yankees_mets,"[dodgers, go, yankees, mets, series, nlcs, wor...",[dodgers beat yankees dodgers defeated two hot...


In [20]:
topic_modelling_df.head()

,No_Stopwords_Text,Topic,Comment_ID
0,yo mlb stop glazing shohei like donut,0,UgyRLpx8l3N4pkd01gV4AaABAg
1,maton sucked every single outing,1,UgyCicep-ndIIZPCwzZ4AaABAg
2,dodgers outscored mets 230 thats win world ser...,1,UgzvfQQwsYqveadG9Y94AaABAg
3,fun series reminded old school baseball,-1,Ugx4z3EZQS0htKNWTA94AaABAg
4,mets match dodgers better team,1,Ugwjom7jkDqGYCHct3p4AaABAg


### Upload Data to AWS S3

In [22]:
from ETL.load.store_data import saveToS3

# saveToS3(all_comments_df_preprocessed, 'youtube-commets-info', 'YouTube_Comments.csv')
saveToS3(videoinfo_df, 'youtube-commets-info', 'Video_Info.csv')
saveToS3(sentiment_result, 'youtube-commets-info', 'Sentiment_Result.csv')
saveToS3(topic_modelling_df, 'youtube-commets-info', 'Topic_Modelling_Result.csv')

Data saved to s3://youtube-commets-info/Video_Info.csv
Data saved to s3://youtube-commets-info/Sentiment_Result.csv
Data saved to s3://youtube-commets-info/Topic_Modelling_Result.csv


### Load data

In [ ]:
from ETL.load.store_data import saveToJson
from ETL.load.store_data import saveToCsv

saveToCsv(all_comments_df_preprocessed, 'data/preprocess_comments.csv')
saveToCsv(videoinfo_df, 'data/video_information.csv')
saveToCsv(sentiment_result, 'data/sentiment_result.csv')
saveToCsv(topic_modelling_df, 'data/topic_modelling_result.csv')


### Visualize the results

In [ ]:
from analyze.visualize import plotSentimentDistribution

plotSentimentDistribution(sentiment_result, 'Sentiment_Label')

In [ ]:
from analyze.visualize import plotWordcloud

plotWordcloud(all_comments_df_preprocessed, 'No_Stopwords_Text')

In [ ]:
from analyze.visualize import visualizeNgram
from analyze.visualize import ngramCount

visualizeNgram(ngramCount(all_comments_df_preprocessed, 'No_Stopwords_Text'), 'count')

In [ ]:
threegram = ngramCount(all_comments_df_preprocessed, 'No_Stopwords_Text', 3)
visualizeNgram(threegram, 'count', 3)

In [ ]:
from analyze.visualize import visualizeLikeCount

visualizeLikeCount(videoinfo_df)

In [ ]:
from analyze.visualize import visualizeReplyCount

visualizeReplyCount(videoinfo_df)

In [ ]:
from analyze.visualize import visualizeViewCount

visualizeViewCount(videoinfo_df)

In [ ]:
from analyze.visualize import visualizeCommentPublishTrend
from analyze.visualize import resamplePublishTime

visualizeCommentPublishTrend(resamplePublishTime(all_comments_df_preprocessed))

In [ ]:
from ETL.transform.preprocess import joinDataFrame
from analyze.visualize import visualizeSemtimentandRTopic

sentiment_model = joinDataFrame(topic_modelling_df, sentiment_result, 'Comment_ID')
visualizeSemtimentandRTopic(    
    sentiment_model,
    topic_column='Topic',
    sentiment_column='Sentiment_Label'
)
